In [1]:
import numpy as np

In [2]:
a = np.array([[1,3], [2,4], [3,5]])

In [3]:
b = np.array([[1,2],[3,1],[4,2],[5,3]])

In [4]:
np.sqrt(np.square(a-b[:,None,:]).sum(2))

array([[1.        , 2.23606798, 3.60555128],
       [2.82842712, 3.16227766, 4.        ],
       [3.16227766, 2.82842712, 3.16227766],
       [4.        , 3.16227766, 2.82842712]])

In [5]:
a

array([[1, 3],
       [2, 4],
       [3, 5]])

In [6]:
a_sq_sum = np.einsum('ik,ik->i',a,a)

b_sq_sum = np.einsum('ik,ik->i',b,b)

a_dot_b = a.dot(b.T)

np.sqrt(a_sq_sum[:,None]-2*a_dot_b+b_sq_sum[None,:])

array([[1.        , 2.82842712, 3.16227766, 4.        ],
       [2.23606798, 3.16227766, 2.82842712, 3.16227766],
       [3.60555128, 4.        , 3.16227766, 2.82842712]])

In [7]:
b_sq_sum.shape

(4,)

In [8]:
a

array([[1, 3],
       [2, 4],
       [3, 5]])

In [9]:
b

array([[1, 2],
       [3, 1],
       [4, 2],
       [5, 3]])

In [20]:
np.einsum('ik,jk->ijk', a,b)

array([[[ 1,  6],
        [ 3,  3],
        [ 4,  6],
        [ 5,  9]],

       [[ 2,  8],
        [ 6,  4],
        [ 8,  8],
        [10, 12]],

       [[ 3, 10],
        [ 9,  5],
        [12, 10],
        [15, 15]]])

In [21]:
import pandas as pd

In [23]:
df = pd.read_clipboard()

In [26]:
df = df.T.reset_index()

In [28]:
df.columns=['name']

In [30]:
df.name.str.split(',')

0             [Emma, F, 20799]
1           [Olivia, F, 19674]
2           [Sophia, F, 18490]
3         [Isabella, F, 16950]
4              [Ava, F, 15586]
5              [Mia, F, 13442]
6            [Emily, F, 12562]
7          [Abigail, F, 11985]
8          [Madison, F, 10247]
9        [Charlotte, F, 10048]
10           [Harper, F, 9564]
11            [Sofia, F, 9542]
12            [Avery, F, 9517]
13        [Elizabeth, F, 9492]
14           [Amelia, F, 8727]
15           [Evelyn, F, 8692]
16             [Ella, F, 8489]
17            [Chloe, F, 8469]
18         [Victoria, F, 7955]
19           [Aubrey, F, 7589]
20            [Grace, F, 7554]
21             [Zoey, F, 7358]
22          [Natalie, F, 7061]
23          [Addison, F, 6950]
24          [Lillian, F, 6869]
25         [Brooklyn, F, 6767]
26             [Lily, F, 6727]
27           [Hannah, F, 6512]
28            [Layla, F, 6428]
29         [Scarlett, F, 5965]
                 ...          
33014          [Zedrick, M, 5]
33015   

In [ ]:
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'data has %d characters, %d unique.' % (data_size, vocab_size)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }


In [1]:
data = open('./data/names.csv', 'r').read() 

In [1]:
import torch

In [2]:
import numpy as np

In [26]:
X = np.random.rand(3,2)

In [27]:
W = np.random.rand(2,4)

In [28]:
Xt = torch.from_numpy(X).float()

In [29]:
Wt = torch.from_numpy(W).float()

In [30]:
Xt.requires_grad=True
Wt.requires_grad=True

In [31]:
scores = torch.mm(Xt,Wt)

In [32]:
l = scores.sum()

In [24]:
Xt.grad

In [25]:
l.backward()

In [13]:
Xt.grad

tensor([[ 0.9203,  1.5850],
        [ 0.9203,  1.5850],
        [ 0.9203,  1.5850]])

In [35]:
Xt.grad_fn

In [33]:
Wt.grad_fn


In [1]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [13]:
lstm = nn.LSTM(3, 53)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

In [14]:
inputs

[tensor([[-0.1473,  0.4982,  0.2940]]),
 tensor([[-0.6479, -0.3016,  0.8846]]),
 tensor([[-2.1708,  1.4156, -1.2350]]),
 tensor([[ 0.4750,  1.0067,  1.1034]]),
 tensor([[ 1.8504, -1.1481, -0.9657]])]

In [23]:
# initialize the hidden state.
hidden = (torch.randn(1, 1, 53),
          torch.randn(1, 1, 53))

In [24]:
hidden

(tensor([[[ 0.0108,  0.9641, -0.0155, -0.4458,  0.6492,  1.0110, -0.1790,
            1.1213, -1.1228,  1.8543,  2.1210, -0.0820,  0.6591, -0.5572,
            0.8680, -0.0472, -1.0035,  1.1797,  1.0145, -0.2112, -0.9302,
           -1.6994, -0.3377,  1.0027, -0.5757,  3.1404, -0.0570, -0.6788,
            1.4388,  0.9930, -0.1493,  0.2475, -0.2305, -1.6334, -1.3680,
            1.8830,  0.7859, -1.2736,  0.1215, -0.8529, -0.2263,  1.5190,
           -0.8346,  0.6603, -0.0719,  0.5490, -1.3864,  1.4419,  2.0726,
           -0.6147,  1.8651,  0.5468,  1.7908]]]),
 tensor([[[ 1.1290,  0.6676,  1.5335, -1.7290,  1.3343,  0.3848, -0.4191,
           -0.4365,  0.1525, -1.9404,  0.8214, -0.4543, -0.8003,  1.0910,
            0.2255,  0.7864, -1.8711, -0.9593, -1.0922, -1.0436, -0.6740,
            1.0460, -0.8622, -0.4517,  0.6310, -1.6984, -0.3481, -0.0629,
            1.1751, -0.0493, -0.2135, -0.0925,  0.7878, -0.8293, -1.3540,
            0.7667, -0.6760,  1.1454, -0.4260, -1.4792, -0.83

In [25]:
i = inputs[0]

In [26]:
i.view(1, 1, -1)

tensor([[[-0.1473,  0.4982,  0.2940]]])

In [27]:
i.view(1, 1, -1).shape

torch.Size([1, 1, 3])

In [28]:
hidden

(tensor([[[ 0.0108,  0.9641, -0.0155, -0.4458,  0.6492,  1.0110, -0.1790,
            1.1213, -1.1228,  1.8543,  2.1210, -0.0820,  0.6591, -0.5572,
            0.8680, -0.0472, -1.0035,  1.1797,  1.0145, -0.2112, -0.9302,
           -1.6994, -0.3377,  1.0027, -0.5757,  3.1404, -0.0570, -0.6788,
            1.4388,  0.9930, -0.1493,  0.2475, -0.2305, -1.6334, -1.3680,
            1.8830,  0.7859, -1.2736,  0.1215, -0.8529, -0.2263,  1.5190,
           -0.8346,  0.6603, -0.0719,  0.5490, -1.3864,  1.4419,  2.0726,
           -0.6147,  1.8651,  0.5468,  1.7908]]]),
 tensor([[[ 1.1290,  0.6676,  1.5335, -1.7290,  1.3343,  0.3848, -0.4191,
           -0.4365,  0.1525, -1.9404,  0.8214, -0.4543, -0.8003,  1.0910,
            0.2255,  0.7864, -1.8711, -0.9593, -1.0922, -1.0436, -0.6740,
            1.0460, -0.8622, -0.4517,  0.6310, -1.6984, -0.3481, -0.0629,
            1.1751, -0.0493, -0.2135, -0.0925,  0.7878, -0.8293, -1.3540,
            0.7667, -0.6760,  1.1454, -0.4260, -1.4792, -0.83

In [29]:
out, hidden = lstm(i.view(1, 1, -1), hidden)

In [30]:
out

tensor([[[ 0.3673,  0.1974,  0.3867, -0.1894,  0.1782,  0.0776, -0.1106,
          -0.0759,  0.0917, -0.2860,  0.2346,  0.3398, -0.0859, -0.0009,
          -0.1181,  0.2300, -0.3513, -0.1808, -0.0921, -0.2064, -0.0829,
           0.4640, -0.0947, -0.1406,  0.2815, -0.4392, -0.0492,  0.0915,
           0.3358,  0.0783, -0.0563, -0.0147,  0.4419, -0.1707, -0.5098,
           0.0686,  0.0099,  0.3897, -0.2950, -0.4322, -0.4559, -0.2449,
           0.0961, -0.4307, -0.3366, -0.1852, -0.1735, -0.1075,  0.1531,
           0.0271,  0.0627, -0.2219,  0.0221]]])

In [31]:
hidden

(tensor([[[ 0.3673,  0.1974,  0.3867, -0.1894,  0.1782,  0.0776, -0.1106,
           -0.0759,  0.0917, -0.2860,  0.2346,  0.3398, -0.0859, -0.0009,
           -0.1181,  0.2300, -0.3513, -0.1808, -0.0921, -0.2064, -0.0829,
            0.4640, -0.0947, -0.1406,  0.2815, -0.4392, -0.0492,  0.0915,
            0.3358,  0.0783, -0.0563, -0.0147,  0.4419, -0.1707, -0.5098,
            0.0686,  0.0099,  0.3897, -0.2950, -0.4322, -0.4559, -0.2449,
            0.0961, -0.4307, -0.3366, -0.1852, -0.1735, -0.1075,  0.1531,
            0.0271,  0.0627, -0.2219,  0.0221]]]),
 tensor([[[ 0.9025,  0.3281,  0.8705, -0.5258,  0.9366,  0.1402, -0.1928,
           -0.2206,  0.2082, -0.7292,  0.2981,  0.5488, -0.1498, -0.0015,
           -0.2559,  0.6147, -1.0476, -0.4644, -0.1595, -0.5448, -0.2217,
            0.9566, -0.1497, -0.3833,  0.6586, -0.9811, -0.2646,  0.3418,
            0.4320,  0.2392, -0.0920, -0.0483,  0.7604, -0.4533, -1.2150,
            0.2456,  0.0253,  0.7643, -0.4498, -0.7378, -0.88

In [2]:
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]])
(tensor([[[-0.3368,  0.0959, -0.0538]]]), tensor([[[-0.9825,  0.4715, -0.0633]]]))


In [45]:
# standard imports
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence,self).__init__()
        self.lstm1 = nn.LSTM(1,64)
        self.lstm2 = nn.LSTM(64,1)
        self.p = 0.5
 
    def forward(self,seq, hc = None):
        out = []
        if hc == None:
            hc1, hc2 = None, None
        else:
            hc1, hc2 = hc
        X_in = torch.unsqueeze(seq[0],0)
        for X in seq.chunk(seq.size(0),dim=0):
            if np.random.rand()>self.p:
                X_in = X
            tmp, hc1 = self.lstm1(X_in,hc1)
            X_in, hc2 = self.lstm2(tmp,hc2)
            out.append(X_in)
        return torch.stack(out).squeeze(1),(hc1,hc2)
 
# Test
lstm = Sequence()
X_seq = torch.tensor(np.random.rand(100,1,1), dtype=torch.float)
lstm_out = lstm(X_seq)

In [50]:
lstm_out[1]

((tensor([[[-0.0445,  0.0783, -0.0575,  0.0463,  0.0175,  0.0080,  0.0249,
            -0.0785, -0.0307, -0.0762, -0.0609, -0.0316, -0.0110, -0.0410,
            -0.0043,  0.0139,  0.0379,  0.0886, -0.0738, -0.0733,  0.0397,
            -0.1035,  0.0033,  0.0569, -0.0927,  0.0353, -0.0167,  0.0190,
             0.0276,  0.0426, -0.0640,  0.0295, -0.0051, -0.0398, -0.0054,
             0.0053, -0.0654, -0.0461,  0.0017,  0.0228, -0.0723, -0.0047,
            -0.0415, -0.0012, -0.0235,  0.0422,  0.0709,  0.0471, -0.0242,
            -0.0387,  0.0315,  0.1014, -0.0630,  0.0535,  0.0032, -0.0651,
             0.1014,  0.0221, -0.0247, -0.0672, -0.0038, -0.0189,  0.0288,
             0.0687]]]),
  tensor([[[-0.0884,  0.1637, -0.1091,  0.0993,  0.0328,  0.0144,  0.0466,
            -0.1557, -0.0654, -0.1720, -0.1278, -0.0574, -0.0223, -0.0781,
            -0.0089,  0.0282,  0.0784,  0.1693, -0.1597, -0.1539,  0.0752,
            -0.2072,  0.0070,  0.1034, -0.2013,  0.0664, -0.0352,  0.0386,


In [37]:
torch.Tensor(np.random(100,1,1))

TypeError: 'module' object is not callable

In [38]:
torch.tensor([1,2,3])

tensor([ 1,  2,  3])

In [40]:
np.random.rand(100,1,1)

array([[[0.50114598]],

       [[0.20965955]],

       [[0.80330871]],

       [[0.9488637 ]],

       [[0.31654119]],

       [[0.42908987]],

       [[0.76255475]],

       [[0.88971279]],

       [[0.2236121 ]],

       [[0.41544046]],

       [[0.08176812]],

       [[0.96080615]],

       [[0.07837588]],

       [[0.7813583 ]],

       [[0.71236155]],

       [[0.61929028]],

       [[0.87964046]],

       [[0.47461922]],

       [[0.34759049]],

       [[0.1074445 ]],

       [[0.2683606 ]],

       [[0.46236363]],

       [[0.62883829]],

       [[0.93127517]],

       [[0.03086925]],

       [[0.04865487]],

       [[0.96027268]],

       [[0.69841834]],

       [[0.56360569]],

       [[0.66332749]],

       [[0.7611323 ]],

       [[0.42187838]],

       [[0.45288885]],

       [[0.52606369]],

       [[0.08862899]],

       [[0.23550386]],

       [[0.90156089]],

       [[0.55168067]],

       [[0.63812317]],

       [[0.32692986]],

       [[0.83661943]],

       [[0.70245

In [53]:
% matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#import gym

In [54]:
lstm1 = nn.LSTM(1,64)
lstm2 = nn.LSTM(64,1)

criterion = nn.MSELoss()
optimizer = optim.Adam(chain(lstm1.parameters(),lstm2.parameters()), lr=0.0001)
for i in range(500):
    data = np.sin(np.linspace(0,5,100)+2*np.pi*np.random.rand())
    xs = data[:-1]
    ys = data[1:]
    X = Variable(torch.Tensor(xs).view(-1,1,1))
    y = Variable(torch.Tensor(ys))
    lstm1.zero_grad()
    lstm2.zero_grad()
    lstm_out, _ = lstm1(X,None)
    lstm_out, _ = lstm2(lstm_out,None)
    loss = criterion(lstm_out,y)
    loss.backward()
    optimizer.step()
    if i%10 == 0:
        print("i {}, loss {}".format(i,loss.data.numpy()[0]))

NameError: name 'chain' is not defined

In [1]:
import numpy as np

In [2]:
np.log(1004)

6.911747300251674